In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

In [4]:
# Downloading data and showing first 5 rows
#Data was scraped by the help of external service
path = 'table-1.csv'
df = pd.read_csv(path)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
# Processing of data
# First of all, we remove all rows where Borough isn't assigned
df.drop(df[df['Borough'] == 'Not assigned'].index,axis=0,inplace=True)
# If Borough is shown, but neighbourhood is unknown, we will consider that Borough = Neighbourhood
df.replace(df[df['Neighbourhood'] == 'Not assigned'].index,df['Borough'][df[df['Neighbourhood'] == 'Not assigned'].index])
# If more than one neighborhood exist in postcode area, we will combine rows
df_new = df.groupby(['Postcode'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
#Remove repeating words in Borough column
newbor = [] #Create a list with clear names
for val in df_new['Borough']:
    val = val.split(' ')
    newbor.append(val[0])
df_new['Borough'] = newbor #Append it instead of old ones
df_new.shape
df_new

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge Malvern
1,M1C,Scarborough,Highland Creek Rouge Hill Port Union
2,M1E,Scarborough,Guildwood Morningside West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village Martin Grove Gardens Richvie...
101,M9V,Etobicoke,Albion Gardens Beaumond Heights Humbergate Jam...
